# AMADEUS Project VIVA Presenation

## ArguMentation-bAsed DEcision sUpport System; a submission for CM2303 One Semester Individual Project by Sh'kyra Jordon

This Jupyter notebook will be presesnted by Sh'kyra Jordon at the project VIVA meeting for the AMADEUS project, scheduled for 9:00 on Tuesday 12th June 2018. This counts towards a final year submission to Cardiff University's School of Copmuter Science and Informatics.

This presentation assumes the AMADEUS report has been read by the audience.

### An initial note
The first note I want to make is that __I did not complete my report__. Some sections were __put together hastily__ in the last few hours before the deadline, when very little sleep had been had and my language skills were deeply hampered. __Not everything makes sense__.

I want to take this presentation as an opportunity to clear up any confusions, to fill in some gaps, and communicate some things that as a result of this project live in my head, but are absent from my report.

## What is AMADEUS?

AMADEUS is a Python 3 based system that aims to serve decision makers.

### The problem

__AMADEUS seeks to aid decision makers in dealing with the contentious nature of inconsistent knowledge bases__:
Any knowledge base that represents real-world information is bound to contain inconsistencies, and thus from this knowledge base, conflicting arguments are inferred. How we deal with these conflicts is ultimately up to a decision maker, but we can create systems to aid this process.

### A solution

Within the report, a process is detailed that
* takes a knowledge base of __logical statements__,
* yields from them all the (infallible) __deductive arguments__ that can be inferred,
* identifies the __conflicts__ within these arguments and thus any counters to every argument, 
* thus producing all the components necessary to create an __argument graph__ of these attacks, which can be passed to third party systems to yield __Dung's extensions__; sets of collectively acceptable arguments that satisfy some condition of *acceptability*.

These sets can be examined by a decision maker, and based on whatever criteria fits her purpose, she can use these extensions, and the nature of their contents, to __choose which arguments to accept__.

### AMADEUS's role in this solution

AMADEUS plays a part in this process by providing the functionality to enable a user to __infer arguments__ from a knowledge base of *simple* logical statements.

### What happens next?

From here, we can demonstrate how AMADEUS can be used with relative ease to __identify the conflicts__ within these arguments, and produce this information in a __common format__ so it can be passed to third party systems that can yield __Dung's extensions__ from it.

## A Demonstration
Using a concrete example, let's run through the main functionality of AMADEUS. To aid this, let us explain the internal structure of AMADEUS.

AMADEUS is made up of 3 modules; `logic.py`, `knowledgebase.py`, and `argumentation.py`:

In [112]:
from logic import Literal, Clause, Rule

- `logic.py` contains the data structures that represent __simple logical statements__, and the components within them; simple clauses and simple rules. Note that these data structures are designed to hold only *simple* logic statements, and cannot be used for more complex statements. 

  Here, a simple clause is a conjunction of simple literals, and a simple rule is an ordered pair where the latter is a set of simple literals treated as its antecedent, and the former is a single simple literal representing a conclusion drawn from its antecedent. Simple literals are the assertion of an atomic logical statement, an *atom*, or the assertion of its logical complement.

  This includes:
  - The `Literal` data structure represents __simple literals__ that are characterised by an `atom` and a sign that depends on whether the literal is positive or negative.
  - The `Clause` data structure represents __simple clauses__, and references `Literal` instances that correspond to the literals it asserts in conjunction.
  - The `Rule` data structure represents __simple rules__, and references `Literal` instances that correspond to the literal it asserts as its consequent (as its `head` attribute), and the literals in its antecedent (as a set of instances in its `body` attribute).

In [113]:
happy, nothappy = Literal("happy", True), Literal("happy", False)  # Literals print in prolog syntax
print("We can create literals {} and {}".format( happy, nothappy ),
      "that consist of the same atom 'happy', but opposite signs.")

We can create literals happy and ~happy that consist of the same atom 'happy', but opposite signs.


In [114]:
sunny, stay_home = Literal("sunny"), Literal("stay_home")  # We can omit signs for positive literals.
c1 = Clause(sunny, stay_home) # Clauses print in prolog syntax
print("Clauses are made up of literals;", c1) 

Clauses are made up of literals; stay_home, sunny.


In [115]:
r1 = Rule(nothappy, sunny, stay_home)  # The first literal is the head, the body literals follow.
print("Rules have a head literal, and body literals;", r1)  # Rules print in prolog sytax

Rules have a head literal, and body literals; ~happy:- sunny, stay_home.


In [116]:
from knowledgebase import KnowledgeBase, PrologString

- `knowledgebase.py` contains data structures to read in and represent __the contents of simple knowledge bases__ as collections of `Clause` and `Rule` (and thus also `Literal`) instances. Here, a simple knowledge base is a collection of simple clauses and rules.
    
  These classes are:
  
  - The `knowledgebase.KnowledgeBase` data structure represents __simple knowledge bases as references to `Clause` and `Rule` instances__ that represent the clauses and rules it asserts.  
  This structure also contains references to the `Case` structures (in `argumentation.py`) that represent the simple cases for each literal within the knowledge base.

  - The `knowledgebase.PrologString` data structure represents __simple knowledge bases in the format of a prolog string__. This structure also provides methods that convert this string into sets of cooresponding `Clause` instances and `Rule` instances.  
  `PrologString` is used by the constructor of a `KnowledgeBase` to convert a string input to the corresponding `logic.py` structures.

In [117]:
print("We can create knowledge bases too;\n", KnowledgeBase(r1, c1))

We can create knowledge bases too;
 ~happy:- sunny, stay_home.
stay_home, sunny.



In [118]:
ps = PrologString("~happy:- stay_home, sunny."  # PrologStrings don't print in prolog syntax
                + "stay_home, sunny. ")
print("Knowledge bases can also be represented as PrologString objects;",
      *ps.clauses.union(ps.rules), sep="\n")
del ps  #cleanup

Knowledge bases can also be represented as PrologString objects;
~happy:- sunny, stay_home.
stay_home, sunny.


In [119]:
from argumentation import Case

- `argumentation.py` contains the data structure `Case` that represents a __simple case__ for a literal (its claim). This structure references a particular `Literal` instance that represents this claim.

  This structure also contains references to collections of `Clause` and `Rule` instances that represent the simple logic statements that form its support.

In [120]:
notwork_well = Literal("work_well", False)
r2 = Rule(notwork_well, stay_home)
kb = KnowledgeBase(r2, c1)
print("For a knowledge base containing:\n", kb)

l = kb._literals_dict["~work_well"]  # Yields kb's instance of notwork_well as l
r = list(l.case.asserting_rules)[0]  # Yields the (only) rule in kb that asserts l, as r
b = list(r.body)[0]  # Yields r's (only) body literal
c = list(b.case.asserting_clauses)[0]  # Yields the (only) clause asserting b, as c
print("The literal {} is asserted by the rule {}".format(l, r),
      "\nThis rule is supported by the clause {}".format(c))

print("\nNote that this is everything you need to get the argument:\n([{}, {}], {})".format(r, c, l))
del l, r, b, c  # cleanup

For a knowledge base containing:
 stay_home, sunny.
~work_well:- stay_home.

The literal ~work_well is asserted by the rule ~work_well:- stay_home. 
This rule is supported by the clause stay_home, sunny.

Note that this is everything you need to get the argument:
([~work_well:- stay_home., stay_home, sunny.], ~work_well)


Nuances of these structures will be mentioned in passing throughout this demonstraction. 

### Creating a knowledge base in simple logic

First we will use `KnowledgeBase` to create a simple knowledge base. For example, with contents:

    ATOMS:
      happy                             := I am happy
      work_well                         := I work well
      sunny                             := It is sunny
      stay_home                         := I stay home
    CLAUSES:
      c1 = sunny, stay_home.            := It was sunny and I stayed home
    RULES:
      r1 = ~happy :- sunny, stay_home.  := If it is sunny and I stay home, I am not happy
      r2 = ~work_well :- stay_home.     := If I stay home, I do not work well
      r3 = happy :- stay_home.          := If I stay home, I am happy
      r4 = work_well :- happy.          := If I am happy, I work well

#### KnowlegdeBase Input

A `KnowledgeBase` reads in a simple logic knowledge base in any of the following formats:

* __a collection of `logic.Clause` and `logic.Rule` objects__; Users can use the data structures provided in the `logic.py` module to represent simple clauses and rules. These data structures can be passed to the `KnowledgeBase` constructor.

In [121]:
r3, r4 = Rule(happy, stay_home), Rule(Literal("work_well"), happy)  # need to create a couple more rules
print("We can populate a KnowledgeBase with the Clause and Rule objects we've made so far (& then some):")
kb = KnowledgeBase(c1, r1, r2, r3, r4)
print("KB:", str(kb).replace("\n", "\n    "))

We can populate a KnowledgeBase with the Clause and Rule objects we've made so far (& then some):
KB: ~happy:- sunny, stay_home.
    stay_home, sunny.
    ~work_well:- stay_home.
    happy:- stay_home.
    work_well:- happy.
    


  Given an input of `Clause` and `Rule` objects, `KnowledgeBase` will *consolidate* these items.
  This means that every such object will be recreated for this `KnowledgeBase` instance, and these duplicates will collectively reference the same `Literal` instances maintained in a hashmap as this `KnowledgeBase`'s `_literals_dict` attribute.

In [122]:
print("Our notwork_well =", notwork_well)
w = kb._literals_dict["~work_well"]  # get kb's literal ~work_well
print("KB's  ~work_well =", w)
print("Shared instance?:", w is notwork_well)

Our notwork_well = ~work_well
KB's  ~work_well = ~work_well
Shared instance?: False


In [123]:
print("Our  r2:", r2)
r = list(w.case.asserting_rules)[0]  # get kb's r2; the only rule that asserts ~work_well
print("KB's r2:", r)
print("Shared instance?:", r is r2)

Our  r2: ~work_well:- stay_home.
KB's r2: ~work_well:- stay_home.
Shared instance?: False


In [124]:
s1 = list(r.body)[0] # get kb's r2's only body literal (stay_home)
print("KB's r2's body literal:", s1)
h = kb._literals_dict["happy"]  # get the literal happy
r = list(h.case.asserting_rules)[0]  # get kb's r3; the only rule that asserts happy
s2 = list(r.body)[0] # get kb's r3's only body literal (stay_home)
print("KB's r3's body literal:", s2)
print("Shared instance?:", s1 is s2)
del w, r, s1, h, s2

KB's r2's body literal: stay_home
KB's r3's body literal: stay_home
Shared instance?: True


* __a string of simple logic clauses and rules in prolog syntax__; Users can create a string to represent simple clauses and rules in prolog syntax. This string can be passed to the `KnowledgeBase` constructor, which will in turn pass it to the `PrologString` constructor to yield sets of corresponding `logic.Clause` and `logic.Rule` objects.

In [125]:
s = "~happy:- sunny, stay_home. ~work_well:- stay_home. happy:- stay_home. work_well:- happy."
s += "stay_home, sunny."
print("We can populate a knowledge base via a prolog-syntax string.",
      "E.G. for the same contents:",
       KnowledgeBase(s), sep="\n")

We can populate a knowledge base via a prolog-syntax string.
E.G. for the same contents:
~happy:- sunny, stay_home.
stay_home, sunny.
~work_well:- stay_home.
happy:- stay_home.
work_well:- happy.



* __the file path of a text-file containing a string of simple logic clauses and rules in prolog syntax__; Users can create a text-file containing a prolog-syntax string as specified in the bullet point above, and pass its file path to the `KnowledgeBase` constructor as a string. The `KnowledgeBase` constructor will assume any single string input passed to it is the contents of a knowledge base either as a file path input, or a prolog syntax string as specified above. Initially it will assume the former format. If it cannot find the file sepcified by this file path, it will assume the latter format.

In [126]:
with open("kb_input_file.txt", "w") as infile:
    infile.write(s)
print("We can populate a knowledge base via a text file containing a prolog-syntax string.",
      "E.G. for the same contents:",
       KnowledgeBase("kb_input_file.txt"), sep="\n")

We can populate a knowledge base via a text file containing a prolog-syntax string.
E.G. for the same contents:
~happy:- sunny, stay_home.
stay_home, sunny.
~work_well:- stay_home.
happy:- stay_home.
work_well:- happy.



### Obtaining a knowledge base's argument

An entailed literal's CCoSE is one possible set of logical arguments that can form the support of an argument for that literal.

We will use our example knowledge base to demonstrate how to find CCoSEs.
    
    KB={sunny, stay_home.             (c1)
        ~happy :- sunny, stay_home.   (r1)
        ~work_well :- stay_home.      (r2)
        happy :- stay_home.           (r3)
        work_well :- happy.}          (r4)

Suppose we had the literal L = `happy`. L is entailed in KB since there exists a rule (r3) in KB with body literals that are either contained, or entailed; `stay_home` is contained since there is a clause (c1) in KB that asserts it.

We can find arguments for L by following this same train of thought, but keeping hold of the evidence for all our justifications at each step. This is an approach similar to backward induction since we work our way backwards through the chain of support from the KB to L.

To find all the CCoSEs of L, given L is entailed, we would follow this algorithm:
    
    def get_CCoSEs_of_literal(L):
        CCoSEs = []
        For all rules r that support L:
            literal_supporting_evidence = defaultdict(list)
            for literals l in r.body:
                if l is contained:
                    literal_supporting_evidence[l] += l.asserting_clauses
                if l is entailed:
                    literal_supporting_evidence[l] += get_CCoSEs_of_literal(l)
            for every permutation p of the elements in literal_supporting_evidence.values():
                CCoSEs.append(p.append(L))
        return CCoSEs        

Every literal in KB is associated with its own `Case`. A `Case`

In [127]:
from itertools import product
from functools import reduce
def rule_supporting_evidence(r):
    """
    Returns a collection (generator) of sets of supporting evidence for Rule r.

    Equivalent to:
    antecedent_evidence_items = [literal_supporting_evidence(l.case) for l in r.body]
    for antecedent_evidence_item in product(*antecedent_evidence_items):
        yield reduce(set().union, antecedent_evidence_item).union({r})
    """
    yield from (reduce(set().union, ccose).union({r})  for ccose in product(*(literal_supporting_evidence(l) for l in r.body)))

In [128]:
from itertools import chain
def literal_supporting_evidence(l):
    """
    Returns a collection (generator) of sets of supporting evidence for the Literal l.
    """
    c = l.case
    # base step: yield each asserting clause of l as a set
    yield from ({clause} for clause in c.asserting_clauses)
    # recursive step: yield each collection of supporting evidence for
    #     (supported) asserted rules of l (as a set)
    yield from chain.from_iterable((rule_supporting_evidence(r) for r in c.supporting_rules))

In [129]:
for l in kb._supported_literals.values():
    print("\nArguments for {}:".format(l))
    supporting_evidence = chain.from_iterable((rule_supporting_evidence(r) for r in l.case.supporting_rules))
    print(*supporting_evidence, sep=" ")


Arguments for ~happy:
{~happy:- sunny, stay_home., stay_home, sunny.}

Arguments for ~work_well:
{stay_home, sunny., ~work_well:- stay_home.}

Arguments for happy:
{stay_home, sunny., happy:- stay_home.}

Arguments for work_well:
{work_well:- happy., stay_home, sunny., happy:- stay_home.}
